In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
import numpy as np
from sqlalchemy import inspect

ModuleNotFoundError: No module named 'geopandas'

In [1]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json
import os

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
db, conn = pgconnect(credentials)

Connected successfully.


In [ ]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.0 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


# TASK 01

## Neighbourhoods.csv

In [ ]:
neighbourhoods = pd.read_csv("Neighbourhoods.csv")
#cleaning
del neighbourhoods['Unnamed: 0']
neighbourhoods.fillna(method = 'ffill', inplace = True)
neighbourhoods["population"] = neighbourhoods["population"].replace(",","", regex = True)
neighbourhoods["number_of_dwellings"] = neighbourhoods["number_of_dwellings"].replace(",","", regex = True)
neighbourhoods["population"] = neighbourhoods["population"].astype(str).astype(np.int64)
neighbourhoods["number_of_dwellings"] = neighbourhoods["number_of_dwellings"].astype(str).astype(np.int64)
neighbourhoods

#business_stats = pd.read_csv('BusinessStats.csv')
#school_catchments = gpd.read_file("school_catchments/catchments_future.shp")

,area_id,area_name,land_area,population,number_of_dwellings,number_of_businesses,median_annual_household_income,avg_monthly_rent,0-4,5-9,10-14,15-19
0,102011028,Avoca Beach - Copacabana,643.8000,7590,2325,738.0,46996.0,1906.0,467,583,604,560
1,102011029,Box Head - MacMasters Beach,3208.6000,10986,3847,907.0,42621.0,1682.0,586,696,661,692
2,102011030,Calga - Kulnura,76795.1000,4841,1575,1102.0,42105.0,1182.0,220,254,304,320
3,102011031,Erina - Green Point,3379.3000,14237,4450,1666.0,43481.0,1595.0,695,778,916,838
4,102011032,Gosford - Springfield,1691.2000,19385,6373,2126.0,45972.0,1382.0,1200,1079,963,977
...,...,...,...,...,...,...,...,...,...,...,...,...
317,106011109,Cessnock Region,1570.4341,7931,3281,673.0,73164.0,1080.0,432,590,577,527
318,106011113,Singleton Region,4067.2349,4919,2055,698.0,87984.0,1000.0,310,394,353,323
319,111021218,Morisset - Cooranbong,330.5208,14959,6298,1154.0,58084.0,1260.0,813,855,828,962
320,114021285,Hill Top - Colo Vale,174.3752,6025,2249,400.0,81120.0,1512.0,443,500,470,451


### Initializing table for Neighbourhoods.csv

In [ ]:
# neighberhoods.to_sql("neighberhoods", con=conn, if_exists='append', index=False)
# query(conn, "select * from neighberhoods")


conn.execute("""
DROP TABLE IF EXISTS neighbourhoods cascade;;
CREATE TABLE neighbourhoods(
   area_id INTEGER PRIMARY KEY,
   area_name VARCHAR(50),
   land_area FLOAT,
   population INTEGER,
   number_of_dwellings INTEGER,
   number_of_businesses INTEGER,
   median_annual_household_income FLOAT,
   avg_monthly_rent FLOAT,
   "0-4" INTEGER,
   "5-9" INTEGER,
   "10-14" INTEGER,
   "15-19" INTEGER  
);""")

neighbourhoods.to_sql("neighbourhoods", con = conn, if_exists='append', index = False)
neighbourhood = query(conn, "select * from neighbourhoods limit 10")
neighbourhood

,area_id,area_name,land_area,population,number_of_dwellings,number_of_businesses,median_annual_household_income,avg_monthly_rent,0-4,5-9,10-14,15-19
0,102011028,Avoca Beach - Copacabana,643.8,7590,2325,738,46996.0,1906.0,467,583,604,560
1,102011029,Box Head - MacMasters Beach,3208.6,10986,3847,907,42621.0,1682.0,586,696,661,692
2,102011030,Calga - Kulnura,76795.1,4841,1575,1102,42105.0,1182.0,220,254,304,320
3,102011031,Erina - Green Point,3379.3,14237,4450,1666,43481.0,1595.0,695,778,916,838
4,102011032,Gosford - Springfield,1691.2,19385,6373,2126,45972.0,1382.0,1200,1079,963,977
5,102011033,Kariong,830.6,6540,1824,352,47896.0,1649.0,472,502,497,591
6,102011034,Kincumber - Picketts Valley,1208.9,7428,1946,479,41642.0,1397.0,404,458,448,416
7,102011035,Narara,770.2,6853,2006,292,46660.0,1512.0,463,470,448,459
8,102011036,Niagara Park - Lisarow,1673.2,8173,2345,477,45762.0,1495.0,525,605,617,606
9,102011037,Point Clare - Koolewong,678.7,6281,2220,312,45966.0,1572.0,371,412,373,343


## BusinessStats.csv

In [ ]:
business_stats = pd.read_csv('BusinessStats.csv')
business_stats.fillna(method = 'ffill', inplace = True)
print(business_stats.dtypes)

area_id                               int64
area_name                            object
number_of_businesses                  int64
accommodation_and_food_services       int64
retail_trade                          int64
agriculture_forestry_and_fishing      int64
health_care_and_social_assistance     int64
public_administration_and_safety      int64
transport_postal_and_warehousing      int64
dtype: object


### Initializing table for BusinessStats.csv

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS business_stats;
CREATE TABLE business_stats(
   area_id INTEGER PRIMARY KEY,
   area_name VARCHAR(50),
   number_of_businesses INTEGER,
   accommodation_and_food_services INTEGER,
   retail_trade INTEGER,
   agriculture_forestry_and_fishing INTEGER,
   health_care_and_social_assistance INTEGER,
   public_administration_and_safety INTEGER,
   transport_postal_and_warehousing INTEGER
);""")
business_stats.to_sql("business_stats", con = conn, if_exists='append', index = False)
query(conn, "select * from business_stats limit 10")

,area_id,area_name,number_of_businesses,accommodation_and_food_services,retail_trade,agriculture_forestry_and_fishing,health_care_and_social_assistance,public_administration_and_safety,transport_postal_and_warehousing
0,101021007,Braidwood,629,26,27,280,11,0,35
1,101021008,Karabar,326,7,10,8,11,0,43
2,101021009,Queanbeyan,724,52,47,11,56,3,77
3,101021010,Queanbeyan - East,580,16,23,4,12,0,57
4,101021011,Queanbeyan Region,1642,39,63,292,34,7,81
5,101021012,Queanbeyan West - Jerrabomberra,885,14,48,14,29,3,59
6,101031013,Bombala,419,18,14,250,9,0,14
7,101031014,Cooma,714,55,47,110,36,0,49
8,101031015,Cooma Region,478,11,15,258,5,0,26
9,101031016,Jindabyne - Berridale,1032,144,59,161,43,3,27


# Combining Neighbourhoods and BusinessStats

In [ ]:
sql = """SELECT *
  FROM neighbourhoods N
      LEFT OUTER JOIN business_stats B ON (B.area_id = N.area_id);"""
cobined_neighbourhood_businessstat = query(conn, sql)
cobined_neighbourhood_businessstat

,area_id,area_name,land_area,population,number_of_dwellings,number_of_businesses,median_annual_household_income,avg_monthly_rent,0-4,5-9,...,15-19,area_id,area_name,number_of_businesses,accommodation_and_food_services,retail_trade,agriculture_forestry_and_fishing,health_care_and_social_assistance,public_administration_and_safety,transport_postal_and_warehousing
0,101051540,Goulburn Region,9035.1221,12670,6817,1528,62712.0,800.0,685,766,...,752,101051540,Goulburn Region,1891,38,52,1041,33,3,81
1,102011028,Avoca Beach - Copacabana,643.8000,7590,2325,738,46996.0,1906.0,467,583,...,560,102011028,Avoca Beach - Copacabana,724,33,35,4,60,0,19
2,102011029,Box Head - MacMasters Beach,3208.6000,10986,3847,907,42621.0,1682.0,586,696,...,692,102011029,Box Head - MacMasters Beach,934,23,45,13,43,3,28
3,102011030,Calga - Kulnura,76795.1000,4841,1575,1102,42105.0,1182.0,220,254,...,320,102011030,Calga - Kulnura,1134,14,43,231,12,0,55
4,102011031,Erina - Green Point,3379.3000,14237,4450,1666,43481.0,1595.0,695,778,...,838,102011031,Erina - Green Point,1712,68,145,19,179,12,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,128021537,Royal National Park,13933.4000,28,4,3,55391.0,1899.0,0,4,...,4,128021537,Royal National Park,3,3,0,0,0,0,0
318,128021538,Sutherland - Kirrawee,775.5000,21760,6924,2086,53378.0,1809.0,1538,1242,...,1033,128021538,Sutherland - Kirrawee,2136,73,113,6,127,17,122
319,128021607,Engadine,895.4000,17424,5219,1121,52667.0,1955.0,1333,1529,...,1031,128021607,Engadine,1106,37,69,3,82,9,51
320,128021608,Loftus - Yarrawarrah,384.4000,7426,2177,417,51243.0,2163.0,503,572,...,447,128021608,Loftus - Yarrawarrah,452,9,20,3,25,5,31


# Reading in Catchment and Break-in Data

In [ ]:
catchments_primary = gpd.read_file("catchments_primary.shp")
catchments_primary

,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,PRIORITY,geometry
0,2838,PRIMARY,Parklea PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((150.93564 -33.71612, 150.93715 -33.7..."
1,3943,PRIMARY,Gordon WPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.14085 -33.74463, 151.14113 -33.7..."
2,2404,PRIMARY,Lindfield EPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.18336 -33.74748, 151.18443 -33.7..."
3,4393,PRIMARY,Carlingford WPS,20220223,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.04518 -33.77303, 151.04526 -33.7..."
4,4615,PRIMARY,Caddies Ck PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((150.92567 -33.72960, 150.92602 -33.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1661,4383,PRIMARY,E A Southee PS,20200315,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((147.94621 -34.55863, 147.95292 -34.5..."
1662,3275,PRIMARY,Tumbarumba PS,20200507,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((148.12885 -35.60082, 148.23155 -35.6..."
1663,2239,PRIMARY,Jindera PS,20200507,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((146.86148 -35.87511, 146.87402 -35.8..."
1664,3594,PRIMARY,Louth PS,20200604,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((145.18403 -29.65805, 145.18434 -29.6..."


In [ ]:
catchments_secondary = gpd.read_file("catchments_secondary.shp")
catchments_secondary

,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,PRIORITY,geometry
0,8503,HIGH_COED,Billabong HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((146.67182 -35.31444, 146.68930 -35.3..."
1,8266,HIGH_COED,James Fallon HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((147.08734 -35.86271, 147.10413 -35.8..."
2,8505,HIGH_COED,Murray HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((146.81448 -35.78341, 146.81250 -35.7..."
3,8246,HIGH_COED,Heathcote HS,None,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((151.03281 -34.02521, 151.03267 -34.0..."
4,8458,HIGH_COED,Kingswood HS,20201016,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"MULTIPOLYGON (((150.68600 -33.74031, 150.68631..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,8213,HIGH_BOYS,Birrong BHS,20211221,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((151.05364 -33.85076, 151.06142 -33.8..."
431,8857,HIGH_GIRLS,GRC Penshurst Girls,20220110,N,N,N,N,N,N,N,Y,Y,Y,Y,N,N,None,"POLYGON ((151.07573 -33.95089, 151.07921 -33.9..."
432,3235,CENTRAL_HIGH,Tooleybuc CS,20200512,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((143.37723 -34.80173, 143.39037 -34.8..."
433,1115,CENTRAL_HIGH,Balranald CS,20200512,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((143.65541 -33.55702, 143.65541 -33.5..."


In [ ]:
catchments_future = gpd.read_file("catchments_future.shp")
catchments_future

,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,geometry
0,2133,PRIMARY,Harbord PS,20200720,2023,2023,2023,2023,2023,2023,2023,0,0,0,0,0,0,"POLYGON ((151.29769 -33.76832, 151.29731 -33.7..."
1,4450,PRIMARY,Berinba PS,20220110,2023,2023,2023,2023,2023,2023,2023,0,0,0,0,0,0,"POLYGON ((149.10301 -34.82127, 149.10167 -34.8..."
2,4683,PRIMARY,Murrumbateman PS,20220110,2023,2023,2023,2023,2023,2023,2023,0,0,0,0,0,0,"POLYGON ((149.10557 -34.89523, 149.11035 -34.8..."
3,4021,PRIMARY,Manly Vale PS,20200805,2023,2023,2023,2023,2023,2023,2023,0,0,0,0,0,0,"POLYGON ((151.27162 -33.76913, 151.27116 -33.7..."
4,8161,HIGH_BOYS,Randwick BHS,20200220,0,0,0,0,0,0,0,2023,2023,2023,2023,2023,2023,"POLYGON ((151.27152 -33.91402, 151.27152 -33.9..."
5,2469,PRIMARY,Manly WPS,20200805,2023,2023,2023,2023,2023,2023,2023,0,0,0,0,0,0,"POLYGON ((151.28453 -33.78790, 151.28559 -33.7..."
6,8463,HIGH_COED,Sylvania HS,20210217,0,0,0,0,0,0,0,2023,2023,2023,2023,2023,2023,"POLYGON ((151.14827 -33.99111, 151.14859 -33.9..."
7,8404,HIGH_COED,Endeavour Sp HS,20210217,0,0,0,0,0,0,0,2023,2023,2023,2023,2023,2023,"POLYGON ((151.12719 -34.01025, 151.12706 -34.0..."
8,3827,PRIMARY,Narraweena PS,20200720,2023,2023,2023,2023,2023,2023,2023,0,0,0,0,0,0,"POLYGON ((151.26725 -33.74242, 151.26789 -33.7..."
9,8555,HIGH_COED,Rose Bay SC,20200220,0,0,0,0,0,0,0,2023,2023,2023,2023,2023,2023,"POLYGON ((151.28072 -33.83287, 151.28095 -33.8..."


In [ ]:
catchments = pd.concat([catchments_primary, catchments_secondary, catchments_future], ignore_index = True)
catchments

,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,PRIORITY,geometry
0,2838,PRIMARY,Parklea PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((150.93564 -33.71612, 150.93715 -33.7..."
1,3943,PRIMARY,Gordon WPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.14085 -33.74463, 151.14113 -33.7..."
2,2404,PRIMARY,Lindfield EPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.18336 -33.74748, 151.18443 -33.7..."
3,4393,PRIMARY,Carlingford WPS,20220223,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.04518 -33.77303, 151.04526 -33.7..."
4,4615,PRIMARY,Caddies Ck PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((150.92567 -33.72960, 150.92602 -33.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,4680,PRIMARY,Googong PS,20220305,2023,2023,2023,2023,2023,2023,2023,0,0,0,0,0,0,NaN,"POLYGON ((149.26653 -35.49838, 149.26703 -35.5..."
2141,2540,PRIMARY,Michelago PS,20220305,2023,2023,2023,2023,2023,2023,2023,0,0,0,0,0,0,NaN,"POLYGON ((149.14237 -35.56872, 149.16075 -35.5..."
2142,8922,HIGH_COED,Jerrabomberra HS,20220305,0,0,0,0,0,0,0,2023,2023,0,0,0,0,NaN,"POLYGON ((149.19808 -35.38147, 149.19852 -35.3..."
2143,8919,HIGH_COED,Bungendore HS,20220305,0,0,0,0,0,0,0,2023,2023,0,0,0,0,NaN,"POLYGON ((149.24085 -34.91439, 149.24367 -34.9..."


In [ ]:
break_ins = gpd.read_file("BreakEnterDwelling_JanToDec2021.shp")
break_ins

,OBJECTID,Contour,Density,ORIG_FID,Shape_Leng,Shape_Area,geometry
0,1,8.0,Low Density,1,0.012138,0.000006,"POLYGON ((149.91078 -37.06636, 149.91080 -37.0..."
1,2,8.0,Low Density,1,0.019106,0.000015,"POLYGON ((149.90601 -37.05837, 149.90602 -37.0..."
2,3,8.0,Low Density,1,0.006068,0.000002,"POLYGON ((148.94250 -37.04209, 148.94253 -37.0..."
3,4,8.0,Low Density,1,0.011006,0.000006,"POLYGON ((149.19583 -36.91833, 149.19584 -36.9..."
4,5,8.0,Low Density,1,0.008073,0.000003,"POLYGON ((149.84836 -36.68459, 149.84838 -36.6..."
...,...,...,...,...,...,...,...
2589,2590,10.0,High Density,3,0.015358,0.000011,"POLYGON ((153.27961 -28.55388, 153.27964 -28.5..."
2590,2591,10.0,High Density,3,0.028327,0.000029,"POLYGON ((150.35939 -28.60530, 150.35941 -28.6..."
2591,2592,10.0,High Density,3,0.006654,0.000002,"POLYGON ((153.56825 -28.38307, 153.56828 -28.3..."
2592,2593,10.0,High Density,3,0.047098,0.000044,"POLYGON ((153.53917 -28.18858, 153.53920 -28.1..."


## Reading in SA2 Shape Data

In [ ]:
sa_two = gpd.read_file("SA2_2016_AUST.shp")
sa_two.drop(sa_two[(sa_two['geometry'] == None)].index, inplace = True)
sa_two

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,geometry
0,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3418.3525,"POLYGON ((149.58423 -35.44427, 149.58444 -35.4..."
1,101021008,11008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,6.9825,"POLYGON ((149.21898 -35.36739, 149.21799 -35.3..."
2,101021009,11009,Queanbeyan,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,4.7634,"POLYGON ((149.21325 -35.34325, 149.21619 -35.3..."
3,101021010,11010,Queanbeyan - East,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,13.0034,"POLYGON ((149.24033 -35.34782, 149.24023 -35.3..."
4,101021011,11011,Queanbeyan Region,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3054.4099,"POLYGON ((149.23580 -35.38738, 149.23771 -35.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,801111141,81141,Namadgi,80111,Urriarra - Namadgi,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,1202.8930,"POLYGON ((148.80406 -35.37620, 148.80416 -35.3..."
2304,901011001,91001,Christmas Island,90101,Christmas Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,136.1356,"POLYGON ((105.67392 -10.41567, 105.67398 -10.4..."
2305,901021002,91002,Cocos (Keeling) Islands,90102,Cocos (Keeling) Islands,901,Other Territories,9OTER,Other Territories,9,Other Territories,13.7163,"MULTIPOLYGON (((96.83047 -12.17636, 96.83045 -..."
2306,901031003,91003,Jervis Bay,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,67.8134,"MULTIPOLYGON (((150.69566 -35.18297, 150.69555..."


# SRID Transformations

In [ ]:
#function borrowed from Week 9 Tutorial to transform Polygon data
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

In [ ]:
srid = 4326
catchments_og = catchments.copy()
catchments['geom'] = catchments['geometry'].apply(lambda x: create_wkt_element(geom = x, srid = srid))
catchments = catchments.drop(columns = ['geometry', 'PRIORITY'])
#drop school years
school_years =  ['KINDERGART', 'YEAR1', 'YEAR2', 'YEAR3', 'YEAR4', 'YEAR5', 'YEAR6', 'YEAR7', 'YEAR8', 'YEAR9', 'YEAR10', 'YEAR11', 'YEAR12']
catchments = catchments.drop(columns = school_years)
catchments

,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,geom
0,2838,PRIMARY,Parklea PS,20181210,MULTIPOLYGON (((150.93563850416004 -33.7161211...
1,3943,PRIMARY,Gordon WPS,20211219,MULTIPOLYGON (((151.14085043012102 -33.7446267...
2,2404,PRIMARY,Lindfield EPS,20211219,MULTIPOLYGON (((151.1833640465581 -33.74748398...
3,4393,PRIMARY,Carlingford WPS,20220223,MULTIPOLYGON (((151.0451821055135 -33.77303212...
4,4615,PRIMARY,Caddies Ck PS,20181210,MULTIPOLYGON (((150.92567327976582 -33.7296030...
...,...,...,...,...,...
2140,4680,PRIMARY,Googong PS,20220305,MULTIPOLYGON (((149.26653458527704 -35.4983768...
2141,2540,PRIMARY,Michelago PS,20220305,MULTIPOLYGON (((149.14237305771536 -35.5687160...
2142,8922,HIGH_COED,Jerrabomberra HS,20220305,MULTIPOLYGON (((149.1980792584528 -35.38147320...
2143,8919,HIGH_COED,Bungendore HS,20220305,MULTIPOLYGON (((149.2408471104619 -34.91439387...


In [ ]:
break_ins_og = break_ins.copy()
break_ins['geom'] = break_ins['geometry'].apply(lambda x: create_wkt_element(geom = x, srid = srid))
break_ins = break_ins.drop(columns = 'geometry')
break_ins

,OBJECTID,Contour,Density,ORIG_FID,Shape_Leng,Shape_Area,geom
0,1,8.0,Low Density,1,0.012138,0.000006,MULTIPOLYGON (((149.91078014700008 -37.0663631...
1,2,8.0,Low Density,1,0.019106,0.000015,MULTIPOLYGON (((149.9060088220001 -37.05836717...
2,3,8.0,Low Density,1,0.006068,0.000002,MULTIPOLYGON (((148.94249939000008 -37.0420853...
3,4,8.0,Low Density,1,0.011006,0.000006,MULTIPOLYGON (((149.1958326340001 -36.91833255...
4,5,8.0,Low Density,1,0.008073,0.000003,MULTIPOLYGON (((149.84836006500007 -36.6845891...
...,...,...,...,...,...,...,...
2589,2590,10.0,High Density,3,0.015358,0.000011,MULTIPOLYGON (((153.27961053500007 -28.5538784...
2590,2591,10.0,High Density,3,0.028327,0.000029,MULTIPOLYGON (((150.3593920080001 -28.60530123...
2591,2592,10.0,High Density,3,0.006654,0.000002,MULTIPOLYGON (((153.56824816100004 -28.3830681...
2592,2593,10.0,High Density,3,0.047098,0.000044,MULTIPOLYGON (((153.5391676050001 -28.18857974...


In [ ]:
sa_two_og = sa_two.copy()
sa_two['geom'] = sa_two['geometry'].apply(lambda x: create_wkt_element(geom = x, srid = srid))
sa_two = sa_two.drop(columns = 'geometry')
sa_two

bad allocation


OSError: exception: access violation reading 0x00000000

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS catchments;
CREATE TABLE catchments (
    "USE_ID" INTEGER,
    "CATCH_TYPE" VARCHAR(20),
    "USE_DESC" VARCHAR(100),
    "ADD_DATE" INTEGER,
    "geom" GEOMETRY(MULTIPOLYGON, 4326)
);""")

conn.execute("""
DROP TABLE IF EXISTS break_ins;
CREATE TABLE break_ins (
    "OBJECTID" INTEGER PRIMARY KEY,
    "Contour" INTEGER,
    "Density" VARCHAR(20),
    "ORIG_FID" INTEGER,
    "Shape_Leng" FLOAT(6),
    "Shape_Area" FLOAT(6),
    "geom" GEOMETRY(MULTIPOLYGON, 4326)
);""")

conn.execute("""
DROP TABLE IF EXISTS sa_two;
CREATE TABLE sa_two (
    "SA2_MAIN16" INTEGER PRIMARY KEY,
    "SA2_5DIG16" INTEGER,
    "SA2_NAME16" VARCHAR(50),
    "SA3_CODE16" INTEGER,
    "SA3_NAME16" VARCHAR(50),
    "SA4_CODE16" INTEGER,
    "SA4_NAME16" VARCHAR(50),
    "GCC_CODE16" VARCHAR(5),
    "GCC_NAME16" VARCHAR(50),
    "STE_CODE16" INTEGER,
    "STE_NAME16" VARCHAR(50),
    "AREASQKM16" FLOAT(4),
    "geom" GEOMETRY(MULTIPOLYGON, 4326)
);""")

In [ ]:
catchments.to_sql("catchments", con = conn, if_exists='append', index = False, dtype = {'geom': Geometry('MULTIPOLYGON', srid)})
query(conn, "SELECT * FROM catchments")

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "catchments_pkey"
DETAIL:  Key ("USE_ID")=(2675) already exists.

[SQL: INSERT INTO catchments ("USE_ID", "CATCH_TYPE", "USE_DESC", "ADD_DATE", geom) VALUES (%(USE_ID)s, %(CATCH_TYPE)s, %(USE_DESC)s, %(ADD_DATE)s, ST_GeomFromEWKT(%(geom)s))]
[parameters: ({'USE_ID': '2838', 'CATCH_TYPE': 'PRIMARY', 'USE_DESC': 'Parklea PS', 'ADD_DATE': '20181210', 'geom': 'SRID=4326;MULTIPOLYGON (((150.93563850416004 -33.716121104794574, 150.9371498209264 -33.718956658615035, 150.9376428817169 -33.71995410858702, 150.93 ... (4392 characters truncated) ... 664420981 -33.71213995553773, 150.93356449231598 -33.71201897714504, 150.93410987263698 -33.71308411804565, 150.93563850416004 -33.716121104794574)))'}, {'USE_ID': '3943', 'CATCH_TYPE': 'PRIMARY', 'USE_DESC': 'Gordon WPS', 'ADD_DATE': '20211219', 'geom': 'SRID=4326;MULTIPOLYGON (((151.14085043012102 -33.74462679248544, 151.14112516446806 -33.74479444119308

In [ ]:
break_ins.to_sql("break_ins", con = conn, if_exists='append', index = False, dtype = {'geom': Geometry('MULTIPOLYGON', srid)})
query(conn, "SELECT * FROM break_ins")

,OBJECTID,Contour,Density,ORIG_FID,Shape_Leng,Shape_Area,geom
0,1,8,Low Density,1,0.012138,0.000006,0106000020E61000000100000001030000000100000015...
1,2,8,Low Density,1,0.019106,0.000015,0106000020E6100000010000000103000000010000001D...
2,3,8,Low Density,1,0.006068,0.000002,0106000020E61000000100000001030000000100000007...
3,4,8,Low Density,1,0.011006,0.000006,0106000020E6100000010000000103000000010000000E...
4,5,8,Low Density,1,0.008073,0.000003,0106000020E6100000010000000103000000010000000F...
...,...,...,...,...,...,...,...
2589,2590,10,High Density,3,0.015358,0.000011,0106000020E61000000100000001030000000100000016...
2590,2591,10,High Density,3,0.028327,0.000029,0106000020E61000000100000001030000000100000028...
2591,2592,10,High Density,3,0.006654,0.000002,0106000020E61000000100000001030000000100000009...
2592,2593,10,High Density,3,0.047098,0.000044,0106000020E6100000010000000103000000010000004D...


In [ ]:
sa_two.to_sql("sa_two", con = conn, if_exists='append', index = False, dtype = {'geom': Geometry('MULTIPOLYGON', srid)})
query(conn, "SELECT * FROM sa_two")

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,geom
0,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3418.3525,0106000020E6100000010000000103000000010000005F...
1,101021008,11008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,6.9825,0106000020E6100000010000000103000000010000002C...
2,101031016,11016,Jindabyne - Berridale,10103,Snowy Mountains,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3939.5483,0106000020E6100000010000000103000000010000005F...
3,101041017,11017,Batemans Bay,10104,South Coast,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,63.7074,0106000020E61000000100000001030000000100000036...
4,101041018,11018,Batemans Bay - South,10104,South Coast,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,30.5948,0106000020E6100000070000000103000000010000001C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2287,801111141,81141,Namadgi,80111,Urriarra - Namadgi,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,1202.8930,0106000020E6100000010000000103000000010000000C...
2288,901011001,91001,Christmas Island,90101,Christmas Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,136.1356,0106000020E61000000100000001030000000100000094...
2289,901021002,91002,Cocos (Keeling) Islands,90102,Cocos (Keeling) Islands,901,Other Territories,9OTER,Other Territories,9,Other Territories,13.7163,0106000020E6100000380000000103000000010000000E...
2290,901031003,91003,Jervis Bay,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,67.8134,0106000020E61000000300000001030000000100000017...


Since there are two different observations for the number of businesses from the neighbourhoods dataset and the business_stats dataset, we've decided to only use the observation from the business_stats dataset since it's not clear what the neighbourhoods dataset defines as a business. The business_stat data set on the other hand, gives us a clear and detailed count for what types of businesses they have included in their count.

In [ ]:
#sql = """create combined1 AS SELECT N.area_id, N.area_name, N.land_area, N.population, N.number_of_dwellings, N.median_annual_household_income,
#N.avg_monthly_rent, N."0-4", N."5-9",N."10-14", N."15-19", B.number_of_businesses, B.accommodation_and_food_services, B.retail_trade, B.agriculture_forestry_and_fishing, B.health_care_and_social_assistance,
#B.public_administration_and_safety, B.transport_postal_and_warehousing
  #FROM neighbourhoods N
      #LEFT OUTER JOIN business_stats B ON (B.area_id = N.area_id);
      
      #CREATE OR REPLACE VIEW combined2 AS SELECT * FROM combined1 C LEFT OUTER JOIN sa_two S on  (C.area_id = S."SA2_MAIN16");
      
      #SELECT *
#FROM combined2 C join break_ins B   ON ST_Contains(C.geom, B.geom);
      
#"""

#sql2 = """SELECT * from neighbourhoods
#"""
#query(conn,sql)

Error encountered: 
(psycopg2.errors.SyntaxError) syntax error at or near "combined1"
LINE 1: create combined1 AS SELECT N.area_id, N.area_name, N.land_ar...
               ^

[SQL: create combined1 AS SELECT N.area_id, N.area_name, N.land_area, N.population, N.number_of_dwellings, N.median_annual_household_income,
N.avg_monthly_rent, N."0-4", N."5-9",N."10-14", N."15-19", B.number_of_businesses, B.accommodation_and_food_services, B.retail_trade, B.agriculture_forestry_and_fishing, B.health_care_and_social_assistance,
B.public_administration_and_safety, B.transport_postal_and_warehousing
  FROM neighbourhoods N
      LEFT OUTER JOIN business_stats B ON (B.area_id = N.area_id);
      
      CREATE OR REPLACE VIEW combined2 AS SELECT * FROM combined1 C LEFT OUTER JOIN sa_two S on  (C.area_id = S."SA2_MAIN16");
      
      SELECT *
FROM combined2 C join break_ins B   ON ST_Contains(C.geom, B.geom);
      
]
(Background on this error at: https://sqlalche.me/e/14/f405)


""


In [ ]:
conn.execute("UPDATE neighbourhood SET young_people = (0-4 + 5-9 + 10-14 + 15-19)")
query(conn, "SELECT * FROM neighbourhood")

ResourceClosedError: This Connection is closed

In [ ]:
conn.close()
db.dispose()